In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

dict_part = {}
list_job = []
for i in range(len(input_)):
    vehicle_page = int(input_.loc[i, 'Vehicle_Page'])
    dict_part[input_.loc[i, 'Url']] = {'vehicle_page': vehicle_page,
                                       'crawler': 0,
                                       'list_vehicle': [],
                                       'vehicle': ''}
    for j in range(vehicle_page):
        list_job.append((i, j))

print('总数量：' + str(len(list_job)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(list_job)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                i, j = list_job[a]

                resp = requests.get(input_.loc[i, 'Url'] + '?page=' + str(j+1),
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =
                
                list_h3 = html.xpath('//section[@id="applications"]/h3[@class="part__titre-groupe-vehicules"]')
                
                # = = = = = = = = = = = = = = =
                
                list_table = html.xpath('//section[@id="applications"]/table[@class="part__table-applications"]')

                # = = = = = = = = = = = = = = =
                
                for h3, table in zip(list_h3, list_table):
                    list_td = table.xpath('./tbody/tr/td[2]')
                    year_min = min([int(td.text.strip()) for td in list_td])
                    year_max = max([int(td.text.strip()) for td in list_td])
                    if year_min == year_max:
                        dict_part[input_.loc[i, 'Url']]['list_vehicle'].append(h3.text.strip() + ' ' + str(year_min))
                    else:
                        dict_part[input_.loc[i, 'Url']]['list_vehicle'].append(h3.text.strip() + ' ' + str(year_min) + '-' + str(year_max))

                dict_part[input_.loc[i, 'Url']]['crawler'] += 1

                # = = = = = = = = = = = = = = =
                
                if dict_part[input_.loc[i, 'Url']]['vehicle_page'] == dict_part[input_.loc[i, 'Url']]['crawler']:
                    dict_part[input_.loc[i, 'Url']]['list_vehicle'] = sorted(dict_part[input_.loc[i, 'Url']]['list_vehicle'])
                    dict_part[input_.loc[i, 'Url']]['vehicle'] = '\n'.join(dict_part[input_.loc[i, 'Url']]['list_vehicle'])

                    # = = = = = = = = = = = = = = =
                    
                    df_temp = pd.DataFrame([{'Url': input_.loc[i, 'Url'],
                                             'Vehicle': dict_part[input_.loc[i, 'Url']]['vehicle']}])

                    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'

                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Url': input_.loc[i, 'Url'],
                                     'Vehicle_Page': input_.loc[i, 'Vehicle_Page']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[i, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Url'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Url'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：2961

[ok] - https://ecat.spectrapremium.com/en/parts/BMP03A
[尝试次数：1] - [剩余数量：2949] - [当前时间：10:56:31]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP06A
[尝试次数：1] - [剩余数量：2948] - [当前时间：10:56:31]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP05A
[尝试次数：1] - [剩余数量：2947] - [当前时间：10:56:31]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP06A
[尝试次数：1] - [剩余数量：2946] - [当前时间：10:56:31]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP06A
[尝试次数：1] - [剩余数量：2945] - [当前时间：10:56:31]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP06A
[尝试次数：1] - [剩余数量：2944] - [当前时间：10:56:31]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP06A
[尝试次数：1] - [剩余数量：2943] - [当前时间：10:56:31]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP09A
[尝试次数：1] - [剩余数量：2942] - [当前时间：10:56:31]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP08A
[尝试次数：1] - [剩余数量：2941] - [当前时间：10:56:31]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP09A
[尝试次数：1] - [剩余数量：2940] - [当前时间：10:56:31]

[ok] - https://ecat.

[ok] - https://ecat.spectrapremium.com/en/parts/CMP01A
[尝试次数：8] - [剩余数量：2863] - [当前时间：10:56:53]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP01A
[尝试次数：2] - [剩余数量：2862] - [当前时间：10:56:54]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP01A
[尝试次数：3] - [剩余数量：2861] - [当前时间：10:56:54]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP01A
[尝试次数：3] - [剩余数量：2860] - [当前时间：10:56:55]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP01A
[尝试次数：3] - [剩余数量：2859] - [当前时间：10:56:55]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP01A
[尝试次数：4] - [剩余数量：2858] - [当前时间：10:56:55]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP02A
[尝试次数：4] - [剩余数量：2857] - [当前时间：10:56:56]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP02A
[尝试次数：1] - [剩余数量：2856] - [当前时间：10:56:56]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP02A
[尝试次数：3] - [剩余数量：2855] - [当前时间：10:56:56]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP02A
[尝试次数：3] - [剩余数量：2854] - [当前时间：10:56:57]

[ok] - https://ecat.spectrapre

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：3] - [剩余数量：2778] - [当前时间：10:57:22]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：4] - [剩余数量：2777] - [当前时间：10:57:23]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：5] - [剩余数量：2776] - [当前时间：10:57:23]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：3] - [剩余数量：2775] - [当前时间：10:57:23]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：5] - [剩余数量：2774] - [当前时间：10:57:24]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：1] - [剩余数量：2773] - [当前时间：10:57:24]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：1] - [剩余数量：2772] - [当前时间：10:57:24]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：2] - [剩余数量：2771] - [当前时间：10:57:25]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：2] - [剩余数量：2770] - [当前时间：10:57:25]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：1] - [剩余数量：2769] - [当前时间：10:57:25]

[ok] - https://ecat.spectrapre

[ok] - https://ecat.spectrapremium.com/en/parts/CMP06A
[尝试次数：2] - [剩余数量：2693] - [当前时间：10:57:50]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP06A
[尝试次数：1] - [剩余数量：2692] - [当前时间：10:57:50]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP06A
[尝试次数：2] - [剩余数量：2691] - [当前时间：10:57:51]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP06A
[尝试次数：1] - [剩余数量：2690] - [当前时间：10:57:51]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP08A
[尝试次数：1] - [剩余数量：2689] - [当前时间：10:57:51]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP06A
[尝试次数：1] - [剩余数量：2688] - [当前时间：10:57:52]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP06A
[尝试次数：3] - [剩余数量：2687] - [当前时间：10:57:52]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP03A
[尝试次数：3] - [剩余数量：2686] - [当前时间：10:57:52]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP08A
[尝试次数：3] - [剩余数量：2685] - [当前时间：10:57:52]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP08A
[尝试次数：3] - [剩余数量：2684] - [当前时间：10:57:53]

[ok] - https://ecat.spectrapre

[ok] - https://ecat.spectrapremium.com/en/parts/CRP05A
[尝试次数：1] - [剩余数量：2608] - [当前时间：10:58:15]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP05A
[尝试次数：1] - [剩余数量：2607] - [当前时间：10:58:15]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP03B
[尝试次数：1] - [剩余数量：2606] - [当前时间：10:58:16]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP03B
[尝试次数：7] - [剩余数量：2605] - [当前时间：10:58:16]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP05A
[尝试次数：2] - [剩余数量：2604] - [当前时间：10:58:16]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP05A
[尝试次数：4] - [剩余数量：2603] - [当前时间：10:58:16]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP05A
[尝试次数：1] - [剩余数量：2602] - [当前时间：10:58:17]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP05A
[尝试次数：1] - [剩余数量：2601] - [当前时间：10:58:17]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP05A
[尝试次数：3] - [剩余数量：2600] - [当前时间：10:58:17]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP05A
[尝试次数：1] - [剩余数量：2599] - [当前时间：10:58:17]

[ok] - https://ecat.spectrapre

...

[ok] - https://ecat.spectrapremium.com/en/parts/SZP03A
[尝试次数：1] - [剩余数量：372] - [当前时间：11:10:54]

[ok] - https://ecat.spectrapremium.com/en/parts/SZP03B
[尝试次数：1] - [剩余数量：371] - [当前时间：11:10:55]

[ok] - https://ecat.spectrapremium.com/en/parts/SZP02A
[尝试次数：6] - [剩余数量：370] - [当前时间：11:10:55]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP01A
[尝试次数：1] - [剩余数量：369] - [当前时间：11:10:55]

[ok] - https://ecat.spectrapremium.com/en/parts/SZP01A
[尝试次数：5] - [剩余数量：368] - [当前时间：11:10:55]

[ok] - https://ecat.spectrapremium.com/en/parts/SZP04A
[尝试次数：4] - [剩余数量：367] - [当前时间：11:10:56]

[ok] - https://ecat.spectrapremium.com/en/parts/SZP02A
[尝试次数：9] - [剩余数量：366] - [当前时间：11:10:56]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP01A
[尝试次数：5] - [剩余数量：365] - [当前时间：11:10:57]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP02B
[尝试次数：2] - [剩余数量：364] - [当前时间：11:10:57]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP04A
[尝试次数：1] - [剩余数量：363] - [当前时间：11:10:58]

[ok] - https://ecat.spectrapremium.com/e

[ok] - https://ecat.spectrapremium.com/en/parts/TOP34A
[尝试次数：2] - [剩余数量：285] - [当前时间：11:11:31]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP33A
[尝试次数：11] - [剩余数量：284] - [当前时间：11:11:31]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP32A
[尝试次数：20] - [剩余数量：283] - [当前时间：11:11:31]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP34A
[尝试次数：4] - [剩余数量：282] - [当前时间：11:11:31]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP33A
[尝试次数：12] - [剩余数量：281] - [当前时间：11:11:31]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP33A
[尝试次数：20] - [剩余数量：280] - [当前时间：11:11:32]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP34B
[尝试次数：1] - [剩余数量：279] - [当前时间：11:11:32]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP33A
[尝试次数：20] - [剩余数量：278] - [当前时间：11:11:33]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP36A
[尝试次数：1] - [剩余数量：277] - [当前时间：11:11:33]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP35A
[尝试次数：1] - [剩余数量：276] - [当前时间：11:11:33]

[ok] - https://ecat.spectrapremium.

[ok] - https://ecat.spectrapremium.com/en/parts/VOP04A
[尝试次数：6] - [剩余数量：199] - [当前时间：11:11:59]

[ok] - https://ecat.spectrapremium.com/en/parts/VOP06A
[尝试次数：6] - [剩余数量：198] - [当前时间：11:11:59]

[ok] - https://ecat.spectrapremium.com/en/parts/VOP09A
[尝试次数：1] - [剩余数量：197] - [当前时间：11:11:59]

[ok] - https://ecat.spectrapremium.com/en/parts/VOP09A
[尝试次数：4] - [剩余数量：196] - [当前时间：11:12:00]

[ok] - https://ecat.spectrapremium.com/en/parts/VOP09A
[尝试次数：6] - [剩余数量：195] - [当前时间：11:12:00]

[ok] - https://ecat.spectrapremium.com/en/parts/VOP09A
[尝试次数：2] - [剩余数量：194] - [当前时间：11:12:01]

[ok] - https://ecat.spectrapremium.com/en/parts/VOP09A
[尝试次数：6] - [剩余数量：193] - [当前时间：11:12:01]

[ok] - https://ecat.spectrapremium.com/en/parts/VOP09A
[尝试次数：5] - [剩余数量：192] - [当前时间：11:12:01]

[ok] - https://ecat.spectrapremium.com/en/parts/VOP09A
[尝试次数：2] - [剩余数量：191] - [当前时间：11:12:01]

[ok] - https://ecat.spectrapremium.com/en/parts/VOP09A
[尝试次数：1] - [剩余数量：190] - [当前时间：11:12:01]

[ok] - https://ecat.spectrapremium.com/e

[ok] - https://ecat.spectrapremium.com/en/parts/VWP28A
[尝试次数：8] - [剩余数量：113] - [当前时间：11:12:24]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP34A
[尝试次数：4] - [剩余数量：112] - [当前时间：11:12:25]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP34A
[尝试次数：2] - [剩余数量：111] - [当前时间：11:12:25]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP34A
[尝试次数：1] - [剩余数量：110] - [当前时间：11:12:25]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP34A
[尝试次数：4] - [剩余数量：109] - [当前时间：11:12:25]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP35A
[尝试次数：1] - [剩余数量：108] - [当前时间：11:12:26]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP35A
[尝试次数：3] - [剩余数量：107] - [当前时间：11:12:26]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP34A
[尝试次数：5] - [剩余数量：106] - [当前时间：11:12:26]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP35A
[尝试次数：1] - [剩余数量：105] - [当前时间：11:12:26]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP26A
[尝试次数：2] - [剩余数量：104] - [当前时间：11:12:27]

[ok] - https://ecat.spectrapremium.com/e

[ok] - https://ecat.spectrapremium.com/en/parts/VWP56A
[尝试次数：1] - [剩余数量：25] - [当前时间：11:12:51]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP56A
[尝试次数：2] - [剩余数量：24] - [当前时间：11:12:51]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP56A
[尝试次数：3] - [剩余数量：23] - [当前时间：11:12:52]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP36A
[尝试次数：6] - [剩余数量：22] - [当前时间：11:12:52]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP57A
[尝试次数：2] - [剩余数量：21] - [当前时间：11:12:53]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP57A
[尝试次数：2] - [剩余数量：20] - [当前时间：11:12:53]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP56A
[尝试次数：10] - [剩余数量：19] - [当前时间：11:12:53]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP56A
[尝试次数：13] - [剩余数量：18] - [当前时间：11:12:53]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP57A
[尝试次数：7] - [剩余数量：17] - [当前时间：11:12:54]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP59A
[尝试次数：1] - [剩余数量：16] - [当前时间：11:12:54]

[ok] - https://ecat.spectrapremium.com/en/parts/